# USA Name Data - kaggle - test

### The code below pulls and summarizes the data from the USA Name Data (BigQuery Dataset) from kaggle.com

In [28]:
# ******************************************************************************
# Dependencies and Setup
# ******************************************************************************
import pandas as pd
from sqlalchemy import create_engine
import credentials

# ******************************************************************************
# File with your personal/local MySQL user name and password in the following 
# format (this file is not stored in repo to protect personal connection info):
# 
# #####################################
# # Enter local MySQL connection info #
# #####################################
# user = '<MySQL user name>'
# password = '<MySQL user password>'
# 
# ******************************************************************************


# ******************************************************************************
# PyMySQL 
# ******************************************************************************
import pymysql
pymysql.install_as_MySQLdb()

# ******************************************************************************
# Provides access to USA Name Data (BigQuery Dataset) from kaggle.com
# ******************************************************************************
from google.cloud import bigquery
client = bigquery.Client()


In [27]:
# ******************************************************************************
# Define the SQL Query to capture all the data including state, year and the 
# total number of births in the USA Name data (from kaggle)
# ******************************************************************************
sql = """
    SELECT state as State_Abbr, year as Birth_Year, SUM(number) as Num_Births
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY Birth_Year, State_Abbr
    ORDER BY State_Abbr, Birth_Year ASC
"""

# ******************************************************************************
# Create a pandas dataframe that captures the result of running the SQL Query 
# above. It includes all 50 states with data from 1910 to 2017 for each state 
# (where it exists).
# ******************************************************************************
usa_name_data_df = client.query(sql).to_dataframe()

# ******************************************************************************
# Print the number of rows in the dataset
# ******************************************************************************
num_rows = usa_name_data_df.shape[0]
print(f"There are {num_rows} rows in this dataset")


There are 5508 rows in this dataset


In [3]:
# ******************************************************************************
# Print a sample of the data included in the dataset
# ******************************************************************************
usa_name_data_df.head()

,State_Abbr,Birth_Year,Num_Births
0,AK,1910,115
1,AK,1911,84
2,AK,1912,141
3,AK,1913,110
4,AK,1914,245


### Connect to local database

In [4]:
rds_connection_string = f'{credentials.user}:{credentials.password}@127.0.0.1/Birth_State_db'
engine = create_engine(f'mysql://{rds_connection_string}')

In [5]:
conn = engine.connect()

### Check for tables

In [6]:
engine.table_names()

['birth_data', 'state_data']

### Use pandas to load USA Name Data (BigQuery Dataset) converted DataFrame into database

In [7]:
usa_name_data_df.to_sql(name='birth_data', con=engine, if_exists='append', index=False)

In [8]:
usa_name_data_df.head()

,State_Abbr,Birth_Year,Num_Births
0,AK,1910,115
1,AK,1911,84
2,AK,1912,141
3,AK,1913,110
4,AK,1914,245


In [9]:

# ******************************************************************************
# Store states csv filepath in a variable
# ******************************************************************************
states_file = "states.csv"

In [10]:
# ******************************************************************************
# Read the Data file with the pandas library
# ******************************************************************************
states_df = pd.read_csv(states_file, encoding="ISO-8859-1")

In [11]:
# ******************************************************************************
# Show just the header to verify we have the data included in the dataframe
# ******************************************************************************
states_df.head()


,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [16]:
renamed_total_births = usa_name_data_df.rename(index=str, columns={"State_Abbr": "Abbreviation"})
renamed_total_births.head()

,Abbreviation,Birth_Year,Num_Births
0,AK,1910,115
1,AK,1911,84
2,AK,1912,141
3,AK,1913,110
4,AK,1914,245


In [17]:
result = renamed_total_births.merge(states_df , on='Abbreviation', how='inner')
result.head()

,Abbreviation,Birth_Year,Num_Births,State
0,AK,1910,115,Alaska
1,AK,1911,84,Alaska
2,AK,1912,141,Alaska
3,AK,1913,110,Alaska
4,AK,1914,245,Alaska


In [ ]:
# Merge two dataframes using an inner join
#merge_table = pd.merge(usa_name_data_df, renamed_state_df, on="State_Abbr")
#merge_table.head()

In [ ]:
#following query(1) is used to calculate the total number of births in each state from year 1910 to 2017

In [30]:
result["Num_Births"] = pd.to_numeric(
   result["Num_Births"])

In [32]:
grouped_state_df = result.groupby(['State'])

state_total = grouped_state_df["Num_Births"].sum()
state_total.head()

State
Alabama        5815853
Alaska          430161
Arizona        3598468
Arkansas       3433745
California    30527811
Name: Num_Births, dtype: int64

In [ ]:
#following query(2) is used to calculate the total number of births in the particular year from 1910 to 2017 for all states

In [37]:
result["Birth_Year"] = pd.to_numeric(
   result["Birth_Year"])

In [49]:
group_by_birth_yr = result.groupby(["Birth_Year"])
group_by_birth_yr.sum()

,Num_Births
Birth_Year,
1910,516314
1911,565822
1912,888001
1913,1028581
1914,1293346
1915,1690016
1916,1786521
1917,1855694
1918,2013354


In [ ]:
#following quer(3) is used to find out total number of births in a year for each state

In [62]:
group_by_birth_yr_statewise= result.groupby(['Birth_Year','State'])
sum_of_births = group_by_birth_yr_statewise["Num_Births"].sum()
sum_of_births

Birth_Year  State               
1910        Alabama                  19694
            Alaska                     115
            Arizona                   1074
            Arkansas                 11042
            California                9164
            Colorado                  3609
            Connecticut               4512
            Delaware                   690
            District of Columbia      1514
            Florida                   6614
            Georgia                  24589
            Hawaii                     572
            Idaho                     1089
            Illinois                 26650
            Indiana                  10161
            Iowa                      7890
            Kansas                    6971
            Kentucky                 15213
            Louisiana                14551
            Maine                     2471
            Maryland                  7003
            Massachusetts            15261
            Michigan 

In [35]:

# ******************************************************************************
# Write the State data to the state_data table in the Birth_State_db MySQL DB
# ******************************************************************************
#states_df.to_sql(name='state_data', con=engine, if_exists='append', index=False)